<a href="https://colab.research.google.com/github/PaixaoThales/tries-automata/blob/main/Notebook_Tries_Automata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação de Bibliotecas

O script utiliza a versão do python 3.12.2

In [1]:
%pip install automathon ipywidgets --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 926.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


### Trie (Árvore de prefixos)

Uma trie ou árvore de prefixos é uma estrutura de dados que armazena cadeias de símbolos de maneira eficiente. Pense que você tem um conjunto de palavras para armazenar como: ['carro', 'carrinho', 'caminhão']. Repare que algumas delas tem um mesmo prefixo, isso já te dá uma dica de que uma boa forma de armazenamento seria algo que guardasse o prefixo de uma só vez. Vale a pena notar que armazenar as palavras em uma lista também pode ser custoso para verificar se uma dada palavra pertence ou não ao conjunto.

Dito isso, veja como uma trie armazenaria aquele conjunto:



```python
type Trie = dict[str, Trie]
def insert(trie: Trie, word: str) -> None:
def search(trie: Trie, prefix: str) -> bool:
```

Vamos discutir abaixo cada um desses métodos.

In [2]:
# Define o tipo Trie para nosso notebook
from typing import NamedTuple, Any

Trie = NamedTuple("Trie", [("root", dict[str, Any])])

#### Trie (Insert Method)

Durante as inserções podemos ler a cadeia de símbolos da palavra e verificar se, a cada símbolo de entrada, já existe uma transição para ele no nosso estado atual, logo temos três casos e usando as palavras *though*, *thought* e *tough* podemos exemplificar:


- Inserindo na árvore vazia (1)

![Alt Text](https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExaTNyOGQ1ZWNoeGpzbzJjOWU4dm9pa2JraXB1MWtpa2JzY2V0bHYzZiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/uZE4FDIKy2lOrAt3pe/giphy.gif)

- Durante a leitura de símbolos encontramos uma transição não mapeada, aproveita o caminho do prefixo e cria um novo nó final (2)

![Alt Text](https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWt4dmR5eTdobDZlaTJ2ZGVxMnJ4aW55eXd2MGRndHU4aHlkb2J4bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AjA6JEoKNMwklWAxb2/giphy.gif)

- Durante a leitura de símbolos todas as transições foram mapeadas, criando um novo ramo (3)

![Alt Text](https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExbmN4Mm92cDQyNDRrdHpxeGx5MXFzMGN0d3FqdzhsMjV5OTg0cGV3ZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xrfh381hD8m3vAYBan/giphy.gif)


In [3]:
def insert(trie: Trie, word: str) -> None:
    current = trie.root
    for letter in word:
        current = current.setdefault(letter, {})
    current.setdefault(".")

#### Trie (Search Method)

Repare que em uma busca (search) podemos processar uma consulta de prefixos, isto é, de acordo com a definição que tivemos na disciplina de autômatos, um prefixo é "qualquer sequência inicial de símbolos de uma palavra" , incluindo a própria palavra. Portanto, a estrutura de dados trie nos fornece duas informações quando executamos uma busca:

- Se uma palavra existe
- Se um prefixo existe

In [4]:
def search(trie: Trie, prefix: str) -> bool:
    node = trie.root
    for letter in prefix:
        if letter not in node:
            return False
        node = node[letter]
    return '.' in node

### Autômatos Finitos Determinísticos (AFD)

Um AFD pode ser representado por uma 5-tupla ordernada da forma (Σ, Q, δ, q0, F), onde:

-  Σ: Conjunto de Símbolos
-  Q: Conjunto de Estados
-  δ: Função de Transição
- q0: Estado Inicial
-  F: Conjunto de Estados Finais

[SIPSER 2005]

Note que é uma estrutura com um formalismo muito maior quando comparada com a Trie, pois precisamos definir um conjunto alfabeto, a função de transição, o estado inicial, o conjunto de estados e um conjunto de estados finais.

Vamos definir seu tipo abaixo:

In [5]:
# Define todos os tipos necessarios para construcao de um AFD
from dataclasses import dataclass, field

@dataclass
class AFD:
    Σ: set[str] = field(default_factory=set)
    Q: set[str] = field(default_factory=set)
    δ: dict[str, dict[str, str]] = field(default_factory=dict)
    q0: str = '0'
    F: set[str] = field(default_factory=set)
    __next__: int = 0

    def __post_init__(self):
        self.add_state()

    def add_state(self) -> None:
        state = self.__next__
        self.δ[str(state)] = {}
        self.__next__ += 1
        return state

    def add_transition(self, state: int, symbol: str, to: int) -> None:
        self.δ[str(state)][symbol] = str(to)

    def add_accept_state(self, state: int) -> None:
        self.F.add(state)


#### AFD (Accept Method)

Um autômato é uma estrutura de processamento muito simples, temos apenas uma operação que indica a aceitação ou não de uma determinada palavra pertencente a uma determinada linguagem e ela tem a seguinte assinatura:

```python
def accept(afd: AFD, word: str) -> bool
```

Abaixo temos a implementação dessa operação. Repare que a aceitação de uma determinada palavra está condicionada ao processamento de toda a palavra por meio de transições disponíveis em __δ__ e o estado de parada pertencer à __F__, conjunto de estados finais do __AFD__.

In [6]:
def accept(afd: AFD, word: str) -> bool:
    actual = afd.q0
    for symbol in word:
      if symbol in afd.δ[actual]:
        actual = afd.δ[actual][symbol]
      else:
        return False
    return actual in afd.F

#### Construindo um AFD

Abaixo vamos construir um AFD com os princípios de construção de uma Trie.

Perceba que todo item da 5-upla ordenada de um AFD pode ser construído enquanto estamos lendo as palavras de uma dada linguagem, sendo assim, o algoritmo de construção da Trie vai ser adaptado para construção do nosso AFD.

Podemos utilizar qualquer conjunto de palavras no código para montar a linguagem.



#### Insert (Method)

Vamos usar essa função como algo muito próximo ao que temos na função insert da Trie, no entanto, perceba que ela irá fazer mais chamadas, pois ao adicionar uma nova palavra para ser aceita pelo autômato uma serie de itens da 5-upla do AFD podem ser alterados.

```python
def insert(afd: AFD, word: str) -> None
```

Abaixo temos a implementação da função que insere uma palavra que deve ser aceita pelo autômato.

In [7]:
def insert(afd: AFD, word: str) -> None:
  state = afd.q0
  for symbol in word:
    next_state = None
    transitions = afd.δ.setdefault(str(state), {})
    if symbol not in transitions:
      next_state = afd.add_state()
      afd.add_transition(state, symbol, next_state)
    else:
      next_state = transitions[symbol]
      state = next_state
  afd.add_accept_state(str(state))

Com a função de inserção feita podemos construir o autômato passando uma linguagem, esta por sua vez é formada por um conjunto de palavras que devem ser aceitas pelo autômato.

In [8]:
def build(language: set[str]) -> AFD:
  afd = AFD()
  for word in language:
        state = afd.q0
        for symbol in word:
            next_state = None
            transitions = afd.δ.setdefault(str(state), {})
            if symbol not in transitions:
                next_state = afd.add_state()
                afd.add_transition(state, symbol, next_state)
            else:
              next_state = transitions[symbol]
            state = next_state
        afd.add_accept_state(str(state))
  return afd

## Construção do Autômato ##

In [9]:
from ipywidgets import interact, Text
from automathon import DFA
from urllib.request import urlopen

def most_common_words() -> set[str]:
  words = set()
  link_file = 'https://raw.githubusercontent.com/PaixaoThales/tries-automata/main/top-1000-commons.txt'
  with urlopen(link_file) as data:
    for line in data:
        words.add(line.decode('utf-8').strip())
  return words

def words() -> set[str]:
  return ['tough', 'though', 'thought', 'thorough', 'through', 'throughout']

def make(option: str) -> None:
  global afd
  if option == "1000 most common words":
    afd = build(most_common_words())
  else:
    afd = build(words())

print("Escolha uma Linguagem (x) Para o Automato Reconhecer:\n")

interact(lambda x: make(x), x=['some words', '1000 most common words']);

Escolha uma Linguagem (x) Para o Automato Reconhecer:



interactive(children=(Dropdown(description='x', options=('some words', '1000 most common words'), value='some …

## Testes ##

In [10]:
from IPython.display import clear_output
try:
  while True:
    clear_output()
    word = input("Insira uma palavra em ingles e verifique se o automato aceita: ")
    if accept(afd, word):
      print(f"A palavra {word} foi aceita.")
    else:
      print(f"A palavra {word} nao foi aceita.")
    option = input("Deseja Continuar com testes (Y/n)? ")
    if option == "n":
      break
except KeyboardInterrupt:
  clear_output()

Insira uma palavra em ingles e verifique se o automato aceita: same
A palavra same nao foi aceita.
Deseja Continuar com testes (Y/n)? n


## Visualização e Minimização do Autômato###

Obs: Não recomendamos gerar a visualização caso o AFD esteja configurado para aceitar as 1000 palavras mais comuns do inglês.

In [11]:
from automathon import DFA

automato = DFA(afd.Q, afd.Σ, afd.δ, afd.q0, afd.F)
automato.view('automata')

## Créditos ##
* Guilherme Rocha Muzi Franco;
* Gustavo Eiji Takata;
* Thales Cunha da Paixão.
